In [13]:
comment  = 'Produto não recomendo não ler em nenhum dispositivo'
comment = comment.lower()


In [14]:
import re
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

Model trained on order_reviews

In [31]:

order_reviews_path = '/Users/sviat/Downloads/OneDrive_1_30-04-2024/S_Data/order_reviews.csv'

#  tokanizer fit
train_data = pd.read_csv(order_reviews_path)
train_data = train_data[['review_comment_message', 'review_score']]
train_data = train_data.dropna()
train_data = train_data.rename(
    columns={'review_comment_message': 'text', 'review_score': 'score'})
train_data["score"] = train_data["score"].astype('category')
train_data["score_label"] = train_data["score"].cat.codes
train_features, train_labels = train_data['text'], tf.one_hot(
    train_data["score_label"], 5)
d = dict(enumerate(train_data["score"].cat.categories))
sentiment = {0:'Negative',1:'Negative',2:'Neutral',3:'Positive',4:'Positive'}

vocab_size = 15000
vector_size = 100
max_seq_len = 20
tokenizer = Tokenizer(oov_token="<OOV>", num_words=vocab_size)
tokenizer.fit_on_texts(train_data['text'])
sequences_train = tokenizer.texts_to_sequences(comment)

padded_train = pad_sequences(
    sequences_train, padding='post', maxlen=max_seq_len)


# path to model
model = tf.keras.models.load_model('models/lstm1.keras')
p = (model.predict(padded_train)> 0.5).astype("int32")
predict = np.array(tf.argmax(p, axis=1))

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 766ms/step


In [34]:
print(comment)
print("score:",d[predict[0]])
print("sentiment:",sentiment[predict[0]])

produto não recomendo não ler em nenhum dispositivo
score: 1
sentiment: Negative


Model trained on dataset from kaggle 
link :  https://www.kaggle.com/datasets/leandrodoze/tweets-from-mgbr

In [35]:
# https://www.kaggle.com/datasets/leandrodoze/tweets-from-mgbr
data_path = '/Users/sviat/SMART/SMART/Tweets_Mg.csv'

#  tokanizer fit
train_data = pd.read_csv(data_path)
train_data = train_data[['Text', 'Classificacao']]
train_data = train_data.dropna()
train_data = train_data.reset_index()
train_data = train_data.rename(
    columns={'Text': 'text', 'Classificacao': 'score'})


for i in range(len(train_data)):
    train_data.loc[i,'text'] = train_data.loc[i,'text'].lower()

for i in range(len(train_data)):
    train_data.loc[i,'text'] = re.sub(
        pattern=r'[\w]*[^\w\s][\w]*', repl=' ', string=train_data.loc[i,'text'])
train_data["score"] = train_data["score"].astype('category')
train_data["score_label"] = train_data["score"].cat.codes
train_features, train_labels = train_data['text'], tf.one_hot(
    train_data["score_label"], 3)
sentiment = {0:'Negative',1:'Neutral',2:'Positive'}


vocab_size = 15000
vector_size = 300
max_seq_len = 32
tokenizer = Tokenizer(oov_token="<OOV>", num_words=vocab_size)
tokenizer.fit_on_texts(train_data['text'])
sequences_train = tokenizer.texts_to_sequences(train_data['text'])

# path to model
model = tf.keras.models.load_model('models/tweets_lstm1.keras')
p = (model.predict(padded_train)> 0.5).astype("int32")
predict = np.array(tf.argmax(p, axis=1))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 354ms/step


In [40]:
print('Comment:',comment)
print('Sentiment:',sentiment[predict[0]])

Comment: produto não recomendo não ler em nenhum dispositivo
Sentiment: Neutral
